In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=KM2XUw_AgKoufxiZUnuf5yy9YomMSrnaH3LIbgdP6H4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/yQFQIiNoundjvPuH1oPgPZQnAceMpJWKc8o14HQVNP4en1CpaA2TxRg

Successfully saved authorization token.


In [9]:
Map = emap.Map(center=[40,-100], zoom=4)
Map.add_basemap('ROADMAP') # Add Google Map
Map

# Load a pre-computed Landsat composite for input.
input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

# Define a region in which to generate a sample of the input.
antioquia = ee.FeatureCollection('users/evaristizabalg/Antioquia')
colombia = ee.FeatureCollection('users/evaristizabalg/Colombia')

# Display the sample region.
Map.center_object(colombia,5)

# Make the training dataset.
training = input.sample(**{
  'region': antioquia,
  'scale': 30,
  'numPixels': 5000
})

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = input.cluster(clusterer).clip(colombia)

# Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')

Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map